### Producing Sentiment variations using LLMs

Author: Raphael Merx
Input: a baseline sentence; Output: variations of this sentence where a target word has more positive or negative sentiment


In [46]:
#pip install simplemind python-dotenv pandas openpyxl
#!pip install openpyxl==3.1.2

In [1]:
from dotenv import load_dotenv
load_dotenv()

from dataclasses import dataclass
from tqdm import tqdm
import simplemind as sm
from typing import Literal, List, get_args
import pandas as pd
import os
import random

random.seed(42)

# Define TARGET_WORD_CHOICES
TARGET_WORD_CHOICES = Literal['abuse', 'anxiety', 'depression', 'mental_health', 'mental_illness', 'trauma']
TARGET_WORD: TARGET_WORD_CHOICES = 'abuse'

# Generate human-readable versions for all target words
TARGET_WORD_HUMAN_CHOICES = {word: word.replace('_', ' ') for word in get_args(TARGET_WORD_CHOICES)}
TARGET_WORD_HUMAN = TARGET_WORD_HUMAN_CHOICES[TARGET_WORD]

# Validate that both 'mental_health' and 'mental_illness' are handled correctly
assert TARGET_WORD_HUMAN_CHOICES['mental_health'] == 'mental health', "Error: 'mental_health' not handled correctly"
assert TARGET_WORD_HUMAN_CHOICES['mental_illness'] == 'mental illness', "Error: 'mental_illness' not handled correctly"

# 1970-1974, ..., 2015-2019
EPOCH_CHOICES = [f"{y}-{y+4}" for y in range(1970, 2020, 5)]
EPOCH = EPOCH_CHOICES[0]

MAX_BASELINES = 10

# can be changed to gemini, see https://pypi.org/project/simplemind/
PROVIDER = "openai"
MODEL = "gpt-4o"

c:\Users\naomi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Get neutral baseline sentences from corpus for LLM input for each target

- **Script 1 Aim**: This script computes sentence-level sentiment scores using the NRC-VAD lexicon for a corpus of target terms spanning 1970–2019. It dynamically determines neutral sentiment ranges for each 5-year epoch by expanding outward from the median sentiment score within the interquartile range (Q1–Q3) until at least 500 sentences are included, capped at 1500. The selected sentences are saved to CSV files for each target and epoch, and a summary file logs the dynamic ranges and counts.

- **Script 2 Aim**: This script processes pre-saved baseline CSV files to calculate sentence counts by year and 5-year epochs for multiple target terms. It generates "year_count_lines.csv" and "epoch_count_lines.csv" summarizing these counts and creates an epoch-based bar plot visualizing sentence distributions across the specified epochs for each target.

In [48]:
#%run step0_get_neutral_baselines_sentiment.py
#%run step1_plot_neutral_baselines_sentiment.py

Setup examples to inject in the prompt

-This code sets up examples of baseline sentences and their intensity-modified variations (more and less intense) to provide context and guidance for the LLM. 
-These examples are formatted into a structured prompt to help the model understand how to generate intensity-modified variations for new sentences.

In [49]:
@dataclass
class Example:
    baseline: str
    positive_sentiment: str
    negative_sentiment: str

    def format_for_prompt(self):
        return f"""<baseline>
{self.baseline}
</baseline>
<positive {TARGET_WORD}>
{self.positive_sentiment}
</positive {TARGET_WORD}>
<negative {TARGET_WORD}>
{self.negative_sentiment}
</negative {TARGET_WORD}>
"""


    @staticmethod
    def read_example_data(word: TARGET_WORD_CHOICES) -> pd.DataFrame:
        # find the sentiment_example_sentences.xlsx file in the `input` folder
        filepath = os.path.join('input', f'sentiment_example_sentences.xlsx')
        df = pd.read_excel(filepath)
        df = df[df['target'] == word]
        print(f"Loaded {len(df)} examples for the term '{word}'")
        return df

example_data = Example.read_example_data(TARGET_WORD)

EXAMPLES = [
    Example(
        baseline=row['baseline'],
        positive_sentiment=row['positive_sentiment'],
        negative_sentiment=row['negative_sentiment']
    )
    for _, row in example_data.iterrows()
]


PROMPT_INTRO = f"""

In psychology research, 'Sentiment' is defined as “a term’s acquisition of a more positive or negative connotation.” This task focuses on the sentiment of the term **<<{TARGET_WORD}>>**.

### **Task**  
You will be given a sentence containing the term **<<{TARGET_WORD}>>**. Your goal is to write two new sentences:
1. One where **<<{TARGET_WORD}>>** has a **more positive connotation**.  
2. One where **<<{TARGET_WORD}>>** has a **more negative connotation**.  

### **Rules**  
1. The term **<<{TARGET_WORD}>>** must remain **exactly as it appears** in the original sentence:
   - Do **not** replace, rephrase, omit, or modify it in any way.
   - Synonyms, variations, or altered spellings are not allowed.  

2. **Meaning and Structure**:  
   - Stay true to the original context and subject matter.  
   - Maintain the sentence’s structure and ensure grammatical accuracy.  

3. **Sentiment Adjustments**:  
   - **Positive Sentiment**: Reflect strengths or benefits realistically, while respecting the potential negativity of **<<{TARGET_WORD}>>**.
   - **Negative Sentiment**: Highlight risks or harms appropriately, avoiding exaggeration or trivialization.  

### **Important**  
- Any response omitting, replacing, or altering **<<{TARGET_WORD}>>** will be rejected.  
- Ensure the output is:  
   - **Grammatically correct**  
   - **Sensitive and serious** in tone  
   - **Free from exaggeration or sensationalism**  

Follow these guidelines strictly to produce valid responses.  
"""


Loaded 5 examples for the term 'abuse'


In [50]:
@dataclass
class SentenceToModify:
    text: str
    positive_variation: str = None
    negative_variation: str = None

    def get_prompt(self):
        prompt = PROMPT_INTRO + "\n\n"
        for example in EXAMPLES:
            prompt += example.format_for_prompt()
            prompt += "\n\n"
        
        prompt += f"""<baseline>
{self.text}
</baseline>
"""
        return prompt
    
    def parse_response(self, response: str):
        # get the sentences inside <positive {TARGET_WORD}> and <negative {TARGET_WORD}>
        self.positive_variation = response.split(f"<positive {TARGET_WORD}>")[1].split(f"</positive {TARGET_WORD}>")[0].strip()
        self.negative_variation = response.split(f"<negative {TARGET_WORD}>")[1].split(f"</negative {TARGET_WORD}>")[0].strip()
        return self.positive_variation, self.negative_variation

    def get_variations(self) -> list[str]:
        """ Returns a list of two strings: one where the TARGET_WORD has a more positive connotation, and one where it has a more negative one. """
        assert TARGET_WORD in self.text.lower(), f"TARGET_WORD {TARGET_WORD} not found in {self.text}"
        prompt = self.get_prompt()
        res = sm.generate_text(prompt=prompt, llm_provider=PROVIDER, llm_model=MODEL)
        return self.parse_response(res)

    @staticmethod
    def load_baselines(word: TARGET_WORD_CHOICES, epoch: EPOCH_CHOICES) -> List[str]:
        # find the baselines.csv file in the `input` folder
        filepath = os.path.join('input', 'baselines', f'{word}_{epoch}.baseline_1500_sentences.csv')
        df = pd.read_csv(filepath)
        # return the `sentence` column as a list
        print(f"Word {word}, epoch {epoch}: ", end="")
        print(f"Loaded {len(df)} baseline sentences, sampling {MAX_BASELINES}")
        baselines = df['sentence'].tolist()
        if MAX_BASELINES and len(baselines) > MAX_BASELINES:
            baselines = random.sample(baselines, MAX_BASELINES)
        baselines = [s.replace(TARGET_WORD, TARGET_WORD) for s in baselines]
        return baselines
    
    @staticmethod
    def save_sentences(sentences: List['SentenceToModify'], word: TARGET_WORD_CHOICES, epoch: EPOCH_CHOICES):
        output_file = os.path.join('output', f'{word}_{epoch}.synthetic_sentences.csv')
        df = pd.DataFrame([{'baseline': s.text, 'positive_variation': s.positive_variation, 'negative_variation': s.negative_variation} for s in sentences])
        df.to_csv(output_file, index=False)
        print(f"Saved {len(sentences)} sentences to {output_file}")
    

baselines = SentenceToModify.load_baselines(TARGET_WORD, EPOCH)
sentence = SentenceToModify(text=baselines[0])
print(sentence.get_prompt())

Word abuse, epoch 1970-1974: Loaded 7 baseline sentences, sampling 10


In psychology research, 'Sentiment' is defined as “a term’s acquisition of a more positive or negative connotation.” This task focuses on the sentiment of the term **<<abuse>>**.

### **Task**  
You will be given a sentence containing the term **<<abuse>>**. Your goal is to write two new sentences:
1. One where **<<abuse>>** has a **more positive connotation**.  
2. One where **<<abuse>>** has a **more negative connotation**.  

### **Rules**  
1. The term **<<abuse>>** must remain **exactly as it appears** in the original sentence:
   - Do **not** replace, rephrase, omit, or modify it in any way.
   - Synonyms, variations, or altered spellings are not allowed.  

2. **Meaning and Structure**:  
   - Stay true to the original context and subject matter.  
   - Maintain the sentence’s structure and ensure grammatical accuracy.  

3. **Sentiment Adjustments**:  
   - **Positive Sentiment**: Reflect strengths or benef

In [55]:
import os
from tqdm import tqdm

# Constants for processing
MAX_BASELINES = 10
OUTPUT_DIR = "output/test"  # Directory where processed files are saved

# Ensure the output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Loop through each target word and epoch
for TARGET_WORD in get_args(TARGET_WORD_CHOICES):
    for EPOCH in EPOCH_CHOICES:
        # Construct the file path to check if it already exists
        output_file = os.path.join(OUTPUT_DIR, f"{TARGET_WORD}_{EPOCH}.synthetic_sentences.csv")
        
        # Print the file path to debug
        print(f"Checking if {output_file} exists...")
        
        # Skip if the file already exists
        if os.path.exists(output_file):
            print(f"Skipping {TARGET_WORD}, {EPOCH}: File already processed.")
            continue
        
        # Load baselines if the file does not exist
        baselines = SentenceToModify.load_baselines(TARGET_WORD, EPOCH)
        sentences = []
        
        # Process each baseline sentence
        for baseline in tqdm(baselines):
            sentence = SentenceToModify(text=baseline)
            try:
                positive_variation, negative_variation = sentence.get_variations()
                sentences.append(sentence)
            except Exception as e:
                print(f"Error processing sentence: {baseline}. Error: {str(e)}")
        
        # Save the processed sentences
        if sentences:  # Only save if there are completed sentences
            SentenceToModify.save_sentences(sentences, word=TARGET_WORD, epoch=EPOCH)
            print(f"Processed and saved: {output_file}")
        else:
            print(f"No valid sentences processed for {TARGET_WORD}, {EPOCH}")

Checking if output/test\abuse_1970-1974.synthetic_sentences.csv exists...
Skipping abuse, 1970-1974: File already processed.
Checking if output/test\abuse_1975-1979.synthetic_sentences.csv exists...
Skipping abuse, 1975-1979: File already processed.
Checking if output/test\abuse_1980-1984.synthetic_sentences.csv exists...
Skipping abuse, 1980-1984: File already processed.
Checking if output/test\abuse_1985-1989.synthetic_sentences.csv exists...
Skipping abuse, 1985-1989: File already processed.
Checking if output/test\abuse_1990-1994.synthetic_sentences.csv exists...
Skipping abuse, 1990-1994: File already processed.
Checking if output/test\abuse_1995-1999.synthetic_sentences.csv exists...
Skipping abuse, 1995-1999: File already processed.
Checking if output/test\abuse_2000-2004.synthetic_sentences.csv exists...
Skipping abuse, 2000-2004: File already processed.
Checking if output/test\abuse_2005-2009.synthetic_sentences.csv exists...
Skipping abuse, 2005-2009: File already processed.


 10%|█         | 1/10 [00:01<00:10,  1.13s/it]

Error processing sentence: The correlation between behavior and anxiety was small but significant.. Error: list index out of range


 20%|██        | 2/10 [00:02<00:11,  1.38s/it]

Error processing sentence: The State-Trait Anxiety Inventory was administered in this study to distinguish A-State and A-Trait subjects, while electromyography was used to investigate qualitative differences in motor behavior.. Error: list index out of range


...

Saved 10 sentences to output\trauma_2005-2009.synthetic_sentences.csv
Processed and saved: output/test\trauma_2005-2009.synthetic_sentences.csv
Checking if output/test\trauma_2010-2014.synthetic_sentences.csv exists...
Word trauma, epoch 2010-2014: Loaded 552 baseline sentences, sampling 10


100%|██████████| 10/10 [00:17<00:00,  1.76s/it]


Saved 10 sentences to output\trauma_2010-2014.synthetic_sentences.csv
Processed and saved: output/test\trauma_2010-2014.synthetic_sentences.csv
Checking if output/test\trauma_2015-2019.synthetic_sentences.csv exists...
Word trauma, epoch 2015-2019: Loaded 627 baseline sentences, sampling 10


100%|██████████| 10/10 [00:20<00:00,  2.03s/it]

Saved 10 sentences to output\trauma_2015-2019.synthetic_sentences.csv
Processed and saved: output/test\trauma_2015-2019.synthetic_sentences.csv
